In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dcase_evaluator import DCASEEvaluator
from utils import (
    load_ss_model,
    calculate_sdr,
    calculate_sisdr,
    parse_yaml,
    get_mean_sdr_from_dict,
)
from models.clap_encoder import CLAP_Encoder
import torch

/fs/classhomes/fall2024/cmsc723/c7230005/micromamba/envs/LASS/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/fs/classhomes/fall2024/cmsc723/c7230005/micromamba/envs/LASS/lib/python3.9/site-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/fs/classhomes/fall2024/cmsc723/c7230005/lass-final-

In [16]:
# dcase_evaluator = DCASEEvaluator(
#     sampling_rate=16000,
#     eval_indexes='lass_synthetic_validation.csv',
#     audio_dir='lass_validation',
# )
# config_yaml='config/audiosep_base.yaml'
# device = "cuda" if torch.cuda.is_available() else "cpu"
# device
# configs = parse_yaml(config_yaml)
# checkpoint_path='checkpoint/audiosep_baseline.ckpt'

# # Load model
# query_encoder = CLAP_Encoder().eval()

# query_encoder
# pl_model = load_ss_model(
#     configs=configs,
#     checkpoint_path=checkpoint_path,
#     query_encoder=query_encoder
# ).to(device)

In [17]:
# type(pl_model.waveform_mixer)

In [10]:
from models.one_peace_encoder import ONE_PEACE_Encoder
import os

os.chdir('ONE-PEACE/')

encoder = ONE_PEACE_Encoder(pretrained_path='/fs/nexus-scratch/vla/finetune_al_retrieval.pt')

/fs/classhomes/fall2024/cmsc723/c7230005/micromamba/envs/LASS/lib/python3.9/site-packages/fairseq/checkpoint_utils.py:317: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  stat

In [14]:
model = encoder.model

In [20]:
# test one-peace
# process raw data
src_tokens = model.process_text(["cow", "dog", "elephant"])
# src_images = model.process_image(["assets/dog.JPEG", "assets/elephant.JPEG"])
src_audios, audio_padding_masks = model.process_audio(["assets/cow.flac", "assets/dog.flac"])

with torch.no_grad():
    # extract normalized features
    text_features = model.extract_text_features(src_tokens)
    # image_features = model.extract_image_features(src_images)
    audio_features = model.extract_audio_features(src_audios, audio_padding_masks)

    # compute similarity
    # i2t_similarity = image_features @ text_features.T
    a2t_similarity = audio_features @ text_features.T

# print("Image-to-text similarities:", i2t_similarity)
print("Audio-to-text similarities:", a2t_similarity)

Audio-to-text similarities: tensor([[0.4051, 0.2017, 0.2907],
        [0.1681, 0.4472, 0.2034]], device='cuda:0')


In [23]:
print(text_features.shape)
text_features

torch.Size([3, 1536])


tensor([[-0.0043,  0.0056, -0.0134,  ...,  0.0168,  0.0266,  0.0100],
        [ 0.0346, -0.0231,  0.0210,  ...,  0.0081, -0.0160, -0.0179],
        [ 0.0415,  0.0005, -0.0117,  ...,  0.0242,  0.0015,  0.0470]],
       device='cuda:0')

In [25]:
print(audio_features.shape)
audio_features

torch.Size([2, 1536])


tensor([[-1.0143e-02, -7.9912e-03, -4.8427e-03,  ..., -1.1590e-02,
          1.5469e-02,  1.6949e-02],
        [ 3.5734e-02, -7.7274e-05,  1.0751e-02,  ...,  1.0404e-02,
          2.4992e-02, -8.9207e-03]], device='cuda:0')